## Demo for Wide and Deep Bandits with Bayesian Linear Regression + Thompson Sampling

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from random import random
from IPython.display import clear_output

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import sys
## path to wide_deep_bandits.py and toy_problem_wu.py files
path = '/content/drive/MyDrive/Fellowship_Deep_and_Wide_Bandit/Wide_and_Deep_Models/'
sys.path.append(path)

#from contextual_dataset_wu import ContextualDataset
from toy_problem_wu import generate_dataframe
from wide_deep_bandits_BLR_TS import Wide_Deep_Bandits_BLR_TS

# Toy problem
The space-bandits toy_problem divides users/customers into two groups

Group 1: age ~25, ARPU ~100, user_id range 0 - 19, best action = 0

Group 2: age ~45, ARPU ~50, user_id range 20 - 39, best action = 2


In [ ]:
## Get training data from toy_problem
n_train_sample = 1000
df = generate_dataframe(n_train_sample)
context_cols = ['age','ARPU']
action_col = ['action']
reward_col = ['reward']
user_id_col = ['user_id']
df.head()

,age,ARPU,action,reward,user_id
0,35.0,27.538680,2,0,36
1,20.0,71.388433,2,0,10
2,38.0,46.973685,2,0,35
3,25.0,88.654907,1,0,10
4,28.0,125.695623,0,10,6


In [ ]:
num_actions = df[action_col].nunique()[0]
num_features = len(context_cols)
num_users = df[user_id_col].nunique()[0]
print("Number of actions:", num_actions)
print("Number of features:", num_features)
print("Number of users:", num_users)

Number of actions: 3
Number of features: 2
Number of users: 40


# Train models

In [ ]:
%%time

### Train models 
demo_model_wide = Wide_Deep_Bandits_BLR_TS(num_actions, num_features, wide_embed_size=num_users, model_type='wide') ## Wide only model
demo_model_deep = Wide_Deep_Bandits_BLR_TS(num_actions, num_features, wide_embed_size=num_users, model_type='deep') ## Deep only model
demo_model_wide_deep = Wide_Deep_Bandits_BLR_TS(num_actions, num_features, wide_embed_size=num_users, model_type='wide_deep') ## Wide + Deep model

for i in range(n_train_sample):
  test_context = df[context_cols].iloc[i].values
  test_action = df[action_col].iloc[i].values
  test_reward = df[reward_col].iloc[i].values
  test_user_id = df[user_id_col].iloc[i].values

  demo_model_wide.update(torch.tensor(df[user_id_col].iloc[i].values), test_context,test_action,test_reward)
  demo_model_deep.update(torch.tensor(df[user_id_col].iloc[i].values), test_context,test_action,test_reward)
  demo_model_wide_deep.update(torch.tensor(df[user_id_col].iloc[i].values), test_context,test_action,test_reward)

clear_output()

CPU times: user 1min 15s, sys: 58.3 s, total: 2min 14s
Wall time: 1min 8s


## Predict Best Action

In [ ]:
user_id = torch.tensor([10])
context = torch.tensor([25.0, 100.0])

## Possible methods
## BLR - Use expected values of Bayesian Linear Regression
## BLR_TS - Use Bayesian Linear Regression + Thompson sampling
## forward - Use forward pass of neural networks
method = 'BLR_TS'

action_wide = demo_model_wide.action(user_id, context, method=method)
action_deep = demo_model_deep.action(user_id, context, method=method)
action_wide_deep = demo_model_wide_deep.action(user_id, context, method=method)

/content/drive/MyDrive/Fellowship_Deep_and_Wide_Bandit/Wide_and_Deep_Models/wide_deep_bandits.py:696: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context = torch.tensor(context).float()


In [ ]:
print("Wide model predicts action: ", action_wide)
print("Deep model predicts action: ", action_deep)
print("Wide + Deep model predicts action: ", action_wide_deep)

Wide model predicts action:  0
Deep model predicts action:  1
Wide + Deep model predicts action:  2


## Get Expected Rewards for Each Action

In [ ]:
## Possible methods
## BLR - Expected values of Bayesian Linear Regression
## forward - Forward pass of neural networks
method = 'BLR'

expected_values_wide = demo_model_wide.expected_values(user_id, context, method=method)
expected_values_deep = demo_model_deep.expected_values(user_id, context, method=method)
expected_values_wide_deep = demo_model_wide_deep.expected_values(user_id, context, method=method)

/content/drive/MyDrive/Fellowship_Deep_and_Wide_Bandit/Wide_and_Deep_Models/wide_deep_bandits.py:696: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context = torch.tensor(context).float()


In [ ]:
print("Wide model expected values: ", expected_values_wide)
print("Deep model expected values: ", expected_values_deep)
print("Wide + Deep model expected values: ", expected_values_wide_deep)

Wide model expected values:  [ 9.93697231 -0.05171538 -0.39681977]
Deep model expected values:  [ -2.66520746   7.52168137 -62.35741814]
Wide + Deep model expected values:  [12.05730841 -1.85155367 37.6499836 ]
